<a href="https://colab.research.google.com/github/sanskriti49/Sentimental-Analysis/blob/main/Sentiment_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Importing libraries

In [1]:
# !git clone https://github.com/sanskriti49/Sentimental-Analysis
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
!pip install nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

###Importing dataset

In [2]:
url = "https://raw.githubusercontent.com/sanskriti49/Sentimental-Analysis/refs/heads/main/Restaurant%20reviews.csv"
df = pd.read_csv(url, encoding="ISO-8859-1")
df = df.drop(columns=["Restaurant","Reviewer","Metadata","Time","Pictures"])



```
# This is formatted as code
```

###Data cleaning

In [3]:
y = df["Rating"]
X = df.drop(columns=["Rating"])
y = y.replace({'Like':3})
y = y.fillna(y.median())
y = pd.to_numeric(y)

y = y.astype(object)
for i in range(0,len(y)):
    y.iloc[i] = round(y.iloc[i],0)

for i in range(0,len(y)):
    if (y[i]>=3):
        y[i] = "Positive"
    else:
        y[i] = "Negative"

In [4]:
#Stemming
ps = PorterStemmer()
corpus = []

for i in range(0, len(X)):
    review = re.sub('[^a-zA-Z]',' ', str(X['Review'][i]))
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

###Data transformation

In [5]:
# Creating Matrix of CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=9000)
X = cv.fit_transform(corpus).toarray()


###Train-Test Split

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [7]:
# Applying MultinomialNB
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB().fit(X_train, y_train)


# Making Predictions and creating confusion matrix

In [8]:
y_pred = classifier.predict(X_test)

# creating confusion matrix
from sklearn.metrics import confusion_matrix
confusion_m = confusion_matrix(y_test, y_pred)
print(confusion_m)

[[ 457  127]
 [ 102 1814]]


#Getting the Accuracy

In [9]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.9084


# Dumping Models

In [10]:
import pickle
pickle.dump(classifier,open('model.pkl','wb'))
pickle.dump(cv,open('cv-model.pkl','wb'))

# Create a DataFrame with cleaned reviews and their corresponding sentiments
final_df = pd.DataFrame({'Review': df['Review'], 'Sentiment': y})

# Display the first few rows of the new DataFrame
print(final_df.head())

                                              Review Sentiment
0  The ambience was good, food was quite good . h...  Positive
1  Ambience is too good for a pleasant evening. S...  Positive
2  A must try.. great food great ambience. Thnx f...  Positive
3  Soumen das and Arun was a great guy. Only beca...  Positive
4  Food is good.we ordered Kodi drumsticks and ba...  Positive


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>